# Treino e teste de modelos treinados com o algortitmo AlphaZero - <b>Ataxx</b>

### - Inspirado no vídeo do freeCodeCamp: https://www.youtube.com/watch?v=wuSQpLinRB4


Imports necessários para o funcionamento do código.

In [1]:
from __future__ import print_function
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import random
import torch
import math
from torch.autograd import Variable
from atax import *

### Implementação da Rede Neural Convolucional (CNN) que será utilizada para a predição de probabilidades de jogadas e valores de estados 

In [2]:
class ResNet(nn.Module):
    def __init__(self, game, num_resBlocks, num_hidden, device):
        super().__init__()
        
        # Inicialização da rede
        self.device = device
        
        # Camada inicial da rede (primeiro bloco)
        self.startBlock = nn.Sequential(
            nn.Conv2d(3, num_hidden, kernel_size=3, padding=1),  # Convolução 2D com ativação ReLU
            nn.BatchNorm2d(num_hidden),  # Normalização por lotes
            nn.ReLU()  # Ativação ReLU
        )
        
        # Bloco principal contendo vários blocos residuais
        self.backBone = nn.ModuleList(
            [ResBlock(num_hidden) for i in range(num_resBlocks)]  # Lista de blocos residuais
        )
        
        # Cabeça de política (saída para ações)
        self.policyHead = nn.Sequential(
            nn.Conv2d(num_hidden, 32, kernel_size=3, padding=1),  # Convolução 2D com ativação ReLU
            nn.BatchNorm2d(32),  # Normalização por lotes
            nn.ReLU(),  # Ativação ReLU
            nn.Flatten(),  # Aplanamento dos dados
            nn.Linear(32 * game.row_count * game.column_count, game.action_size)  # Camada totalmente conectada
        )
        
        # Saída para avaliação de estado
        self.valueHead = nn.Sequential(
            nn.Conv2d(num_hidden, 3, kernel_size=3, padding=1),  # Convolução 2D com ativação ReLU
            nn.BatchNorm2d(3),  # Normalização por lotes
            nn.ReLU(),  # Ativação ReLU
            nn.Flatten(),  # Aplanamento dos dados
            nn.Linear(3 * game.row_count * game.column_count, 1),  # Camada fully connected
            nn.Tanh()  # Função de ativação tangente hiperbólica
        )
        
        # Configuração do dispositivo
        self.to(device)
        
    def forward(self, x):
        # Propagação dos dados através da rede
        x = self.startBlock(x)
        for resBlock in self.backBone:
            x = resBlock(x)
        policy = self.policyHead(x)  # Saída da cabeça de política
        value = self.valueHead(x)  # Saída da cabeça de valor
        return policy, value


class ResBlock(nn.Module):
    def __init__(self, num_hidden):
        super().__init__()
        # Definição do bloco residual
        self.conv1 = nn.Conv2d(num_hidden, num_hidden, kernel_size=3, padding=1)  # Primeira camada convolucional
        self.bn1 = nn.BatchNorm2d(num_hidden)  # Normalização por lotes para a primeira camada convolucional
        self.conv2 = nn.Conv2d(num_hidden, num_hidden, kernel_size=3, padding=1)  # Segunda camada convolucional
        self.bn2 = nn.BatchNorm2d(num_hidden)  # Normalização por lotes para a segunda camada convolucional
        
    def forward(self, x):
        # Propagação dos dados através do bloco residual
        residual = x
        x = F.relu(self.bn1(self.conv1(x)))  # Ativação ReLU após a primeira camada convolucional
        x = self.bn2(self.conv2(x))  # Normalização por lotes após a segunda camada convolucional
        x += residual  # Adição do atalho (conexão residual)
        x = F.relu(x)  # Ativação ReLU final
        return x

### Implementação do algoritmo MCTS (Monte Carlo Tree Search) que será utilizado para a escolha de jogadas
Neste caso é o MCTS Paralelo, que utiliza múltiplas threads para simular vários jogos simultaneamente durante o treino para escolher a melhor jogada possível

In [3]:
# Classe auxiliar para armazenar os dados de um jogo
class Node:
    def __init__(self, game, args, state,parent=None, action_taken=None, prior=0, visit_count=0):
        # Inicialização de um nó no MCTS
        self.game = game
        self.args = args
        self.state = state
        self.parent = parent
        self.action_taken = action_taken
        self.prior = prior
        
        self.children = []
        
        self.visit_count = visit_count
        self.value_sum = 0
        
    def is_fully_expanded(self):
        return len(self.children) > 0
    
    def select(self):
        # Seleção do melhor filho com base no UCB (Upper Confidence Bound)
        best_child = None
        best_ucb = -np.inf
        
        for child in self.children:
            ucb = self.get_ucb(child)
            if ucb > best_ucb:
                best_child = child
                best_ucb = ucb
                
        return best_child
    
    def get_ucb(self, child):
        # Cálculo do UCB para um filho específico
        if child.visit_count == 0:
            q_value = 0
        else:
            q_value = 1 - ((child.value_sum / child.visit_count) + 1) / 2
        return q_value + self.args['C'] * (math.sqrt(self.visit_count) / (child.visit_count + 1)) * child.prior
    
    def expand(self, policy):
        # Expansão do nó com base na policy de probabilidade
        child=None
        for action, prob in enumerate(policy):

            if prob > 0:
                child_state = self.state.copy()
                child_state = self.game.get_next_state(child_state, action, 1)
                child_state = self.game.change_perspective(child_state, player=-1)

                child = Node(self.game, self.args, child_state, self, action, prob)
                self.children.append(child)
        if(child is None):
            child_state = self.state.copy()
            child_state = self.game.change_perspective(child_state, player=-1)

            child = Node(self.game, self.args, child_state, self, action, prob)
            self.children.append(child)   
        return child
            
    def backpropagate(self, value):
        # Backpropagation do valor do nó até a raiz
        self.value_sum += value
        self.visit_count += 1
        
        value = self.game.get_opponent_value(value)
        if self.parent is not None:
            self.parent.backpropagate(value)  


class MCTSParallel:
    def __init__(self, game, args, model):
        # Inicialização do MCTS
        self.game = game
        self.args = args
        self.model = model
        
    @torch.no_grad()
    def search(self, state,spGames):
        # Realiza uma busca MCTS para vários jogos simultaneamente
        policy, _ = self.model(
            torch.tensor(self.game.get_encoded_state(state), device=self.model.device)
        )
        policy = torch.softmax(policy, axis=1).cpu().numpy()
        policy = (1 - self.args['dirichlet_epsilon']) * policy + self.args['dirichlet_epsilon'] \
            * np.random.dirichlet([self.args['dirichlet_alpha']] * self.game.action_size, size=policy.shape[0])
        
        # Inicializa os nós raiz para cada jogo
        for i, spg in enumerate(spGames):
            spg_policy = policy[i]
            valid_moves = self.game.get_valid_moves(state[i],1)
            spg_policy *= valid_moves
            if( np.sum(spg_policy)!=0):
                spg_policy /= np.sum(spg_policy)
            spg.root = Node(self.game, self.args, state[i], visit_count=1)
            spg.root.expand(spg_policy)
        
        for search in range(self.args['num_searches']):
            for spg in spGames:
                spg.node = None
                node = spg.root
                while node.is_fully_expanded():
                    node = node.select()

                # Verifica se o jogo terminou
                value, is_terminal = self.game.get_value_and_terminated(node.state)
                value = self.game.get_opponent_value(value)

                if is_terminal:
                    node.backpropagate(value)
                    
                else:
                    spg.node = node
            # Realiza a expansão dos nós
            expandable_spGames = [mappingIdx for mappingIdx in range(len(spGames)) if spGames[mappingIdx].node is not None]
            
            # Realiza a expansão dos nós
            if len(expandable_spGames) > 0:
                state = np.stack([spGames[mappingIdx].node.state for mappingIdx in expandable_spGames])
                
                policy, value = self.model(
                    torch.tensor(self.game.get_encoded_state(state), device=self.model.device) # Codifica o estado do jogo
                )

                # Decodifica a policy e o valor
                policy = torch.softmax(policy, axis=1).cpu().numpy()
                value = value.cpu().numpy()

            # Atualiza os nós expandidos
            for i, mappingIdx in enumerate(expandable_spGames):
                node = spGames[mappingIdx].node
                spg_policy, spg_value = policy[i], value[i]
                
                valid_moves = self.game.get_valid_moves(node.state,1)
                spg_policy *= valid_moves
                if np.sum(spg_policy)!=0:
                    spg_policy /= np.sum(spg_policy)

                node.expand(spg_policy)
                node.backpropagate(spg_value)  
            
            

MCTS normal para testar os modelos treinados

In [4]:
class MCTS:
    def __init__(self, game, args, model):
        # Inicialização do MCTS
        self.game = game
        self.args = args
        self.model = model
        
    @torch.no_grad()
    def search(self, state):
        # Busca MCTS para selecionar a melhor ação dado um estado do jogo
        root = Node(self.game, self.args, state, visit_count=1)
        
        # Obtenção da política de probabilidade a partir da rede
        policy, _ = self.model(
            torch.tensor(self.game.get_encoded_state(state), device=self.model.device).unsqueeze(0)
        )
        policy = torch.softmax(policy, axis=1).squeeze(0).cpu().numpy()

        # Adição de ruído de Dirichlet para exploração estocástica
        policy = (1 - self.args['dirichlet_epsilon']) * policy + self.args['dirichlet_epsilon'] \
            * np.random.dirichlet([self.args['dirichlet_alpha']] * self.game.action_size)

        # Aplicação das jogadas válidas ao nó raiz
        valid_moves = self.game.get_valid_moves(root.state,1)
        policy *= valid_moves
        policy /= np.sum(policy)
        root.expand(policy)

        # Realização de iterações de busca MCTS
        for search in range(self.args['num_searches']):
            node = root
            while node.is_fully_expanded():
                node = node.select()
            
            # Avaliação do valor e se é um state terminal
            value, is_terminal = self.game.get_value_and_terminated(node.state)
            value = self.game.get_opponent_value(value)

            if not is_terminal:
                # Obtenção da policy de probabilidade e valor da rede para o nó atual
                policy, value = self.model(
                    torch.tensor(self.game.get_encoded_state(node.state), device=self.model.device).unsqueeze(0)
                )
                policy = torch.softmax(policy, axis=1).squeeze(0).cpu().numpy()
                
                # Aplicação das jogadas válidas ao nó atual
                valid_moves = self.game.get_valid_moves(node.state,1)
                policy *= valid_moves
                policy /= np.sum(policy)
                value = value.item()
                
                # Expansão do nó e backpropagation do valor
                node.expand(policy)
                
            node.backpropagate(value)    
            
        # Cálculo das probabilidades de ação normalizadas a partir dos visit_count dos filhos do nó raiz
        action_probs = np.zeros(self.game.action_size)
        for child in root.children:
            action_probs[child.action_taken] = child.visit_count
        action_probs /= np.sum(action_probs)
        return action_probs

### Implementação do algoritmo AlphaZero que utiliza a CNN e o MCTS para treinar um modelo de IA para jogar Ataxx

Neste caso é o AlphaZero Paralelo, que utiliza múltiplas threads para simular diversos jogos ao mesmo tempo

In [5]:
from tqdm.notebook import trange
class AlphaZeroParallel:
    def __init__(self, model, optimizer, game, args):
        # Inicialização do AlphaZero para treino paralelo
        self.model = model
        self.optimizer = optimizer
        self.game = game
        self.args = args
        self.mcts = MCTSParallel(game, args, model)
        
    def selfPlay(self):
        # Simulação de partidas para recolha de dados de treino
        return_memory = []
        player = 1
        spGames = [SPG(self.game) for spg in range(self.args['num_parallel_games'])]
        
        # Realização de jogadas até que todos os jogos terminem
        while len(spGames) > 0:
            print(len(spGames))
            states = np.stack([spg.state for spg in spGames])
            neutral_states = self.game.change_perspective(states, player)
            #passar variavel de passar e tabuleiro antigo##########################################
            action_probs = self.mcts.search(neutral_states,spGames)

            # Realização de jogadas para cada jogo
            for i in range(len(spGames))[::-1]:
                spg = spGames[i]
                
                action_probs = np.zeros(self.game.action_size)
                for child in spg.root.children:
                    action_probs[child.action_taken] = child.visit_count
                
                
                if np.sum(action_probs)!=0:
                    action_probs /= np.sum(action_probs)

                spg.memory.append((spg.root.state, action_probs, player))
                # Realização de jogadas aleatórias com base na temperatura
                temperature_action_probs = action_probs ** (1 / self.args['temperature'])
                if np.sum(temperature_action_probs)!=0:
                    temperature_action_probs /= np.sum(temperature_action_probs)
                action = np.random.choice(self.game.action_size, p=temperature_action_probs) # Divide temperature_action_probs with its sum in case of an error

                spg.state = self.game.get_next_state(spg.state, action, player)
                value, is_terminal = self.game.get_value_and_terminated(spg.state)
                # Adição dos dados de treino
                if is_terminal:
                    for hist_neutral_state, hist_action_probs, hist_player in spg.memory:
                        hist_outcome = value if hist_player == player else self.game.get_opponent_value(value)
                        return_memory.append((
                            self.game.get_encoded_state(hist_neutral_state),
                            hist_action_probs,
                            hist_outcome
                        ))
                    del spGames[i]
            
            player = self.game.get_opponent(player)
        
        return return_memory
                
    def train(self, memory):
        # Treino da rede neuronal com os dados recolhidos
        random.shuffle(memory)
        for batchIdx in range(0, len(memory), self.args['batch_size']):
            sample = memory[batchIdx:min(len(memory) - 1, batchIdx + self.args['batch_size'])] # Change to memory[batchIdx:batchIdx+self.args['batch_size']] in case of an error
            state, policy_targets, value_targets = zip(*sample)
            
            state, policy_targets, value_targets = np.array(state), np.array(policy_targets), np.array(value_targets).reshape(-1, 1)
            
            state = torch.tensor(state, dtype=torch.float32, device=self.model.device)
            policy_targets = torch.tensor(policy_targets, dtype=torch.float32, device=self.model.device)
            value_targets = torch.tensor(value_targets, dtype=torch.float32, device=self.model.device)
            
            out_policy, out_value = self.model(state)
            
            policy_loss = F.cross_entropy(out_policy, policy_targets)
            value_loss = F.mse_loss(out_value, value_targets)
            loss = policy_loss + value_loss
            
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
    
    def learn(self):
        # Treino iterativo do modelo de acordo com os parâmetros definidos
        for iteration in range(self.args['num_iterations']):
            memory = []
            
            self.model.eval()
            for selfPlay_iteration in trange(self.args['num_selfPlay_iterations'] // self.args['num_parallel_games']):
                memory += self.selfPlay()
                
            self.model.train()
            for epoch in trange(self.args['num_epochs']):
                self.train(memory)
            
            # Guarda o modelo e o optimizer
            torch.save(self.model.state_dict(), f"model_{iteration}_{self.game}.pt")
            torch.save(self.optimizer.state_dict(), f"optimizer_{iteration}_{self.game}.pt")
class SPG:
    def __init__(self, game):
        # Inicialização de uma instância de jogo para autojogo
        self.state = game.get_initial_state()
        self.memory = []
        self.root = None
        self.node = None

### Classe auxiliar para a criação de um tabuleiro de Ataxx e para a realização de jogadas

In [19]:
import atax
import numpy as np

class Atax():
    def __init__(self,n):
        self.row_count = n
        self.column_count = n
        # tamanho do vetor de ações
        self.action_size = n*n*24
        # dicionario de ações/posições  
        # cada peça tem 24 ações possiveis, logo o tamanho de ações possíveis é no máximo 24*n*n
        # mesmo que algumas ações não sejam possiveis, o número de ações possíveis é sempre menor que 24*n*n
        # exemplo: peças nos cantos não podem ir para fora do tabuleiro
        self.num_to_pos= { 0 :(-2,-2),1 :(-2,-1),2 :(-2,0),3:(-2,1),4:(-2,2),
                            5:(-1,-2),6:(-1,-1),7:(-1,0),8:(-1,1),9:(-1,2),
                            10:(0,-2),11:(0,-1),12:(0,1),13:(0,2),
                            14:(1,-2),15:(1,-1),16:(1,0),17:(1,1),18:(1,2),
                            19:(2,-2),20:(2,-1),21:(2,0),22:(2,1),23:(2,2)}
        self.pos_to_num= {v: k for k, v in self.num_to_pos.items()}
        
    def __repr__(self):
        return "Atax"
    
    # retorna o estado inicial, tudo a 0 menos as peças dos cantos que são 1 e -1
    def get_initial_state(self):
        b=np.zeros((self.row_count, self.column_count))
        b[0][0]=1
        b[0][self.column_count-1]=-1
        b[self.row_count-1][self.column_count-1]=1
        b[self.row_count-1][0]=-1
        return b
    
    # retorna o novo estado depois de uma ação
    def get_next_state(self, state, action, player):
        #para o caso de passar
        if action ==-1:
            return state
        b = atax.State(state,player)
        # val é o valor da ação no dicionario
        val = action//24 
        # xi e yi são as coordenadas da peça
        xi= val//self.column_count
        yi=val%self.column_count
        # pos é a posição para onde a peça vai
        pos= self.num_to_pos[action%24]
        # xf e yf são as coordenadas da posição final
        xf = xi+pos[0]
        yf = yi+pos[1]
        # ty é o tipo de movimento 
        ty=1
        # se for um movimento de captura
        if abs(pos[0])==2 or abs(pos[1])==2:
            ty=2
        # executa o movimento
        move= atax.Move(xi,yi,xf,yf,player,ty)
        boa=b.execute_move(move)
        return boa
    
    # retorna as jogadas validas
    def get_valid_moves(self, state,player):
        valid_moves = [0] * self.action_size
        b = atax.State(state,player)
        possi=b.available_moves(1)
        # para cada jogada possivel calcula o valor da ação no vetor de ações e coloca a 1 no vetor de jogadas validas
        for i in possi:
            dicti=(i.xf - i.xi,i.yf-i.yi)
            action = self.pos_to_num[dicti] + (i.xi*self.column_count + i.yi)*24
            valid_moves[action]=1
        return valid_moves
    
    # retorna o valor e se o jogo terminou
    def get_value_and_terminated(self, state):
        value,terminated = self.winner(state)
        return value, terminated
    
    # retorna o vencedor e se o jogo terminou
    def winner(self, state):
        b=atax.State(state,1)
        pecas= self.count(state)
        # verifica os available moves para cada jogador
        b1=b.available_moves(1).size
        b_1=b.available_moves(-1).size
        # se um dos jogadores não tiver jogadas termina o jogo
        if pecas[0] == 0:
            # se tiverem as mesmas peças o jogo termina em empate
            if pecas[1] == pecas[2]:
                return 0,True
            # se o jogador -1 tiver mais peças ganha
            elif pecas[1] < pecas[-1]:
                return -1,True
            # se o jogador 1 tiver mais peças ganha
            else: return 1,True
        # se um dos jogadores não tiver peças termina o jogo
        elif pecas[1] == 0:
            return -1,True
        elif pecas[-1] == 0:
            return 1,True
        
        b1=b.available_moves(1).size
        b_1=b.available_moves(-1).size
        
        # se um dos jogadores não tiver jogadas termina o jogo
        if b1==0:
            if pecas[1] < pecas[-1] + pecas[0]:
                return -1,True
            else: return 1,True
        if b_1==0:
            if pecas[-1] < pecas[1] + pecas[0]:
                return 1,True
            else: return -1,True
        return 0,False
    
    # conta o numero de peças de cada jogador
    def count(self,state):
        pecas=[0,0,0]
        for i in range(0, self.row_count):
            for j in range(0, self.column_count):
                if state[i][j] == 0:
                    pecas[0]+=1
                elif state[i][j] == 1:
                    pecas[1] +=1
                else:
                    pecas[-1] +=1
        return pecas

    
    # retorna o jogador oposto
    def get_opponent(self, player):
        return -player
    
    # retorna o valor do jogador oposto
    def get_opponent_value(self, value):
        return -value
    
    # retorna o estado com a perspetiva do jogador
    def change_perspective(self, state, player):
        return state * player
    
    # retorna o estado codificado
    def get_encoded_state(self, state):
        encoded_state = np.stack(
            (state == -1, state == 0, state == 1)
        ).astype(np.float32)
        
        if len(state.shape) == 3:
            encoded_state = np.swapaxes(encoded_state, 0, 1)
        
        return encoded_state

##  Treino de um modelo capaz de jogar Ataxx

In [7]:

# Inicialização do jogo
game = Atax(5)

# Definição do dispositivo a utilizar
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialização do modelo
model = ResNet(game, 9, 128, device)

# Definição do optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)

# Definição dos parâmetros de treino
args = {
    'C': 2,
    'num_searches': 200,
    'num_iterations': 5,
    'num_selfPlay_iterations':100,
    'num_parallel_games': 25,
    'num_epochs': 4,
    'batch_size': 128,
    'temperature': 1.25,
    'dirichlet_epsilon': 0.25,
    'dirichlet_alpha': 0.3
}

# Inicialização do AlphaZero e treino
alphaZero = AlphaZeroParallel(model, optimizer, game, args)
#alphaZero.learn()

### Teste com os modelos treinados

Teste 1: Modelo com menos treino vs Modelo com mais treino

In [12]:
n_board = 5
game = Atax(n_board)
player = 1
device = torch.device("cpu")

# Inicia o modelo 1, para o jogador 1
model = ResNet(game, 9, 128, device)
model.load_state_dict(torch.load("./Modelos/5x5_model_0_Atax.pt", map_location=torch.device('cpu')))
model.eval()

mcts = MCTS(game, args, model)

# Inicia o modelo 2, para o jogador -1
model2 = ResNet(game, 9, 128, device)
model2.load_state_dict(torch.load("./Modelos/5x5_model_1_Atax.pt", map_location=torch.device('cpu')))
model2.eval()

mcts2 = MCTS(game, args, model2)

state = game.get_initial_state()
ata = atax.State(state,1)


while True:
    print(state)

    if player ==1:
        neutral_state = game.change_perspective(state, player)
        mtcs_probs = mcts2.search(neutral_state)
        action = np.argmax(mtcs_probs)
        print("bot 1- ", str(action))
    
    else:
        neutral_state = game.change_perspective(state, player)
        mtcs_probs = mcts.search(neutral_state)
        action = np.argmax(mtcs_probs)
        print("bot 2- ", str(action))
        
    state = game.get_next_state(state, action, player)

    value, terminated = game.get_value_and_terminated(state)
    
    if terminated:
        print(state)
        print("Game over")
        if value != 0:
            print(f"{value} wins")
        else:
            print("Draw")
        break
    
    player = game.get_opponent(player)
                            

[[ 1.  0.  0.  0.  0. -1.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  0.  1.]]
bot 1-  840
[[ 1.  0.  0.  0.  0. -1.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  0.  0.]]
bot 2-  134
[[ 1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  0.  0.]]
bot 1-  515
[[ 1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  0.  0.]]
bot 2-  723
[[ 1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0. -1. -1.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
bot 1-  505
[[ 1.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0. -1. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.

Teste 2: Modelo vs Humano

In [18]:
n_board = 5
game = Atax(n_board)
player = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model = ResNet(game, 9,128,device)
model.load_state_dict(torch.load("./Modelos/5x5_model_1_Atax.pt"))
model.eval()

mcts = MCTS(game, args, model)

state = game.get_initial_state()
ata = atax.State(state,1)

while True:
    print(state)

    if player ==1:
        neutral_state = game.change_perspective(state, player)
        mtcs_probs = mcts.search(neutral_state)
        action = np.argmax(mtcs_probs)
        print("bot 1- ", str(action))

    else:
        b = atax.State(state,player)
        valid_moves = b.available_moves(player)
        
        for move in valid_moves:
            dicti = (move.xf - move.xi, move.yf - move.yi)
            action = game.pos_to_num[dicti] + (move.xi*game.column_count + move.yi)*24
            print(f"From: ({move.xi}, {move.yi}) To: ({move.xf}, {move.yf}). Write : {action}")
        action = int(input("action: "))


    state = game.get_next_state(state, action, player)

    value, terminated = game.get_value_and_terminated(state)

    if terminated:
        print(state)
        print("Game over")
        if value != 0:
            print(f"{value} wins")
        else:
            print("Draw")
        break

    player = game.get_opponent(player)

[[ 1.  0.  0.  0.  0. -1.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  0.  1.]]


bot 1-  840
[[ 1.  0.  0.  0.  0. -1.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  0.  0.]]
From: (0, 5) To: (0, 3). Write : 130
From: (0, 5) To: (0, 4). Write : 131
From: (0, 5) To: (1, 3). Write : 134
From: (0, 5) To: (1, 4). Write : 135
From: (0, 5) To: (1, 5). Write : 136
From: (0, 5) To: (2, 3). Write : 139
From: (0, 5) To: (2, 4). Write : 140
From: (0, 5) To: (2, 5). Write : 141
From: (5, 0) To: (3, 0). Write : 722
From: (5, 0) To: (3, 1). Write : 723
From: (5, 0) To: (3, 2). Write : 724
From: (5, 0) To: (4, 0). Write : 727
From: (5, 0) To: (4, 1). Write : 728
From: (5, 0) To: (4, 2). Write : 729
From: (5, 0) To: (5, 1). Write : 732
From: (5, 0) To: (5, 2). Write : 733
[[ 1.  0.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  0.  0.]]
bot 1-  515
[[ 1.  0.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0

ValueError: invalid literal for int() with base 10: ''